In [1]:
import nltk
import string
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('wordnet')
from nltk.stem import 	WordNetLemmatizer

#importing layers,models and optimizers along with callbacks from Keras

from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
import codecs
import collections

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/marlontabone/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# loading .txt file 
path = '1.txt'
text = open(path).read().lower() # reading text
print('Corpus length:', len(text)) #printing length of text
#print(text)
nltk.download('punkt')

Corpus length: 69670


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/marlontabone/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
def clean_doc(doc):

     clean_words= []

     words = word_tokenize(text)
    
     for word in words:
           word = word.strip(string.punctuation)
           if len(word)>=1 and word.isdigit()==False and word not in ("äì","\n","Äô","\n\n",'\u2009','\xa0','n’t',',','.','\n  \n\n  \n',' \n\n','“','\n\n  \n',':','-'):
                    word = word.lower()
                    clean_words.append(word)
                        
     return clean_words

token=clean_doc(text)

In [4]:
wd = WordNetLemmatizer()
newDict = []
for w in token:
        newDict.append(wd.lemmatize(w))
vocab_size = len(newDict)
#print(newDict)
print(vocab_size)

12426


In [5]:
sequence_len=30
seq=[]
for i in range(0,len(newDict)-sequence_len):
    seq.append(newDict[i:i+sequence_len])

In [6]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(seq)
sequence = tokenizer.texts_to_sequences(seq)

In [7]:
#print(sequence[2])

print(len(sequence))
import numpy as np
arr=np.array(sequence)

X, Y = arr[:,:-1], arr[:,-1]
Y=to_categorical(Y,num_classes=vocab_size)

12396


In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,Embedding

seq_length = X.shape[1]

model = Sequential()
model.add(Embedding(vocab_size, 150, input_length=seq_length))
model.add(LSTM(1024, return_sequences=True))
model.add(LSTM(1024)) 
model.add(layers.Dropout(0.2)) # dropout layer 
model.add(Dense(1024, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 29, 150)           1863900   
_________________________________________________________________
lstm (LSTM)                  (None, 29, 1024)          4812800   
_________________________________________________________________
lstm_1 (LSTM)                (None, 1024)              8392704   
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              1049600   
_________________________________________________________________
dense_1 (Dense)              (None, 12426)             12736650  
Total params: 28,855,654
Trainable params: 28,855,654
Non-trainable params: 0
____________________________________________

In [9]:
model.compile(loss='categorical_crossentropy',
optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X, Y, batch_size=80, epochs=150)

Epoch 1/150
155/155 [==============================] - 559s 4s/step - loss: 7.0024 - accuracy: 0.0487
Epoch 2/150
155/155 [==============================] - 561s 4s/step - loss: 6.3133 - accuracy: 0.0569
Epoch 3/150
155/155 [==============================] - 587s 4s/step - loss: 6.1226 - accuracy: 0.0665
Epoch 4/150
155/155 [==============================] - 567s 4s/step - loss: 5.9788 - accuracy: 0.0725
Epoch 5/150
155/155 [==============================] - 770s 5s/step - loss: 5.8470 - accuracy: 0.0810
Epoch 6/150
155/155 [==============================] - 471s 3s/step - loss: 5.9363 - accuracy: 0.0770
Epoch 7/150
155/155 [==============================] - 501s 3s/step - loss: 5.8801 - accuracy: 0.0742
Epoch 8/150
155/155 [==============================] - 580s 4s/step - loss: 5.6999 - accuracy: 0.0813
Epoch 9/150
130/155 [========================>.....] - ETA: 1:32 - loss: 5.5697 - accuracy: 0.0865

### Length of extracted word sequences
maxlen = 30

# We sample a new sequence every `step` words
step = 1

# This holds our extracted sequences
sentences = []

# This holds the targets (the follow-up words)
next_words = []

for i in range(0, len(token) - maxlen, step):
    sentences.append(token[i: i + maxlen])
    next_words.append(token[i + maxlen])

In [223]:
import string
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenised_text = sentences[rand.randrange(len(sentences))]
seed_text = TreebankWordDetokenizer().detokenize(tokenised_text)

def generate_seq(model,tokenizer,seq_length,seed_text,n_words):

     result = []
     in_text =seed_text
     for i in range(n_words):
          encoded = tokenizer.texts_to_sequences([in_text])[0]
          encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
          yhat = model.predict_classes(encoded)
          out_word = ''
          for word,index in tokenizer.word_index.items():
               if index==yhat:
                  out_word = word
                  break
          in_text+=' ' + out_word
          result.append(word)
     return ' '.join(result)


generated = generate_seq(model, tokenizer, seq_length, seed_text, 50)
#print("Seed:"+seed_text)
print(generated)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/marlontabone/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


found yearsold worked dig assorted desk tall testified court morning prime minister whenever leader labour party gonzi left next glass herman public decide time guilty mepa electoral past micallef refused ferry came jeffrey pullicino orlando left bartolo seriously sant evarist labour general pullicino orlando even eye pullicino orlando challenging sant
